# param-init

感觉对这个模型的使用还是很生，只知道了大体的过程。今天学一下怎么自动初始化参数模型。

In [1]:
import numpy as np
import torch
from torch import nn

In [4]:
net1=nn.Sequential(
    nn.Linear(30,40),
    nn.ReLU(),
    nn.Linear(40,50),
    nn.ReLU(),
    nn.Linear(50,10)
)

In [7]:
w1=net1[0].weight
b1=net1[0].bias
print(w1)

Parameter containing:
-0.0535  0.0829  0.0784  ...   0.0960 -0.0120  0.0591
-0.1428  0.1791  0.0588  ...   0.1737 -0.0497 -0.0645
 0.1438  0.1387  0.0505  ...   0.0693  0.0817  0.0894
          ...             ⋱             ...          
-0.0631  0.0044  0.1578  ...  -0.0851 -0.0106 -0.1699
-0.0895 -0.1646 -0.1489  ...  -0.0211 -0.0144 -0.0165
 0.0547  0.0959 -0.0380  ...   0.0475  0.0234  0.0513
[torch.FloatTensor of size 40x30]



net.weight.data可以赋值，用tensor类型赋值

In [9]:
net1[0].weight.data=torch.from_numpy(np.random.uniform(3,5,size=(40,30)))
print(net1[0].weight)

Parameter containing:
 4.9939  3.3346  4.1735  ...   4.2062  4.3881  3.8105
 3.9910  4.5676  3.3322  ...   4.6725  4.0798  3.8372
 3.5112  4.9891  4.2573  ...   3.5003  3.4784  4.9560
          ...             ⋱             ...          
 3.7719  3.4358  3.0875  ...   4.3238  4.5484  3.5932
 3.8501  3.8932  4.0726  ...   4.9563  3.0169  4.7087
 4.3198  4.3361  3.7862  ...   4.1611  4.9641  3.7846
[torch.DoubleTensor of size 40x30]



module的参数初始化，children与modules属性

In [12]:
 class nets(nn.Module):
    def __init__(self):
        super(nets,self).__init__()
        self.l1=nn.Sequential(
            nn.Linear(30,40),
            nn.ReLU()
        )
        self.l1[0].weight.data=torch.randn(40,30)
        self.l2=nn.Sequential(
            nn.Linear(40,50),
            nn.ReLU()
        )
        self.l3=nn.Sequential(
            nn.Linear(50,10),
            nn.ReLU()
        )
        
    def forward(self,x):
        x=self.l1(x)
        x=self.l2(x)
        x=self.l3(x)
        return x

In [13]:
net2=nets()

In [16]:
for i in net2.children():
    print i

Sequential(
  (0): Linear(in_features=30, out_features=40, bias=True)
  (1): ReLU()
)
Sequential(
  (0): Linear(in_features=40, out_features=50, bias=True)
  (1): ReLU()
)
Sequential(
  (0): Linear(in_features=50, out_features=10, bias=True)
  (1): ReLU()
)


In [19]:
times=0
for i in net2.modules():
    times=times+1
    print ("{}th layer".format(times))
    print i
    

1th layer
nets(
  (l1): Sequential(
    (0): Linear(in_features=30, out_features=40, bias=True)
    (1): ReLU()
  )
  (l2): Sequential(
    (0): Linear(in_features=40, out_features=50, bias=True)
    (1): ReLU()
  )
  (l3): Sequential(
    (0): Linear(in_features=50, out_features=10, bias=True)
    (1): ReLU()
  )
)
2th layer
Sequential(
  (0): Linear(in_features=30, out_features=40, bias=True)
  (1): ReLU()
)
3th layer
Linear(in_features=30, out_features=40, bias=True)
4th layer
ReLU()
5th layer
Sequential(
  (0): Linear(in_features=40, out_features=50, bias=True)
  (1): ReLU()
)
6th layer
Linear(in_features=40, out_features=50, bias=True)
7th layer
ReLU()
8th layer
Sequential(
  (0): Linear(in_features=50, out_features=10, bias=True)
  (1): ReLU()
)
9th layer
Linear(in_features=50, out_features=10, bias=True)
10th layer
ReLU()


modules会访问到Sequential 中的每一层

# torch.nn.init

torch.nn.init是torch内置的一种初始化方法，内置了许多方法，比如xavier,xavier方法初始化使每一层方差差不多相等.

In [21]:
from torch.nn import init
print (net1[0].weight)

Parameter containing:
 4.9939  3.3346  4.1735  ...   4.2062  4.3881  3.8105
 3.9910  4.5676  3.3322  ...   4.6725  4.0798  3.8372
 3.5112  4.9891  4.2573  ...   3.5003  3.4784  4.9560
          ...             ⋱             ...          
 3.7719  3.4358  3.0875  ...   4.3238  4.5484  3.5932
 3.8501  3.8932  4.0726  ...   4.9563  3.0169  4.7087
 4.3198  4.3361  3.7862  ...   4.1611  4.9641  3.7846
[torch.DoubleTensor of size 40x30]



In [22]:
init.xavier_uniform(net1[0].weight)

Parameter containing:
-0.0212 -0.0344 -0.2210  ...  -0.2592  0.0384  0.0736
 0.2183 -0.0952  0.1968  ...  -0.1184  0.2106 -0.0716
 0.0197 -0.2801 -0.0715  ...   0.0121 -0.2503 -0.0377
          ...             ⋱             ...          
-0.1674  0.1011  0.1595  ...   0.2303 -0.2367  0.1490
-0.1702 -0.2873 -0.1055  ...   0.2096  0.1936  0.2441
 0.2132 -0.2794 -0.0246  ...  -0.2487  0.0955 -0.0741
[torch.DoubleTensor of size 40x30]

参数已经被修改。